In [1]:
import os
from time import perf_counter
import numpy as np
import pandas as pd

In [2]:
# Reads in Kaggle .csv file of NHL stats and performs initial cleaning
def load_data():
    names = ["game_skater_stats", "game_plays", "game_shifts", "game"]
    t2 = perf_counter()
    df = {}

    print("load")
    for name in names:
        df[name] = pd.read_csv(f"kaggle_stats/{name}.csv").drop_duplicates(ignore_index=True)
        t1, t2 = t2, perf_counter()
        print(f"{name:>25}: {t2 - t1:.4g} sec, {len(df[name])} rows")
        # return a dict of df
    return df

In [3]:
# Breaks NHL dataframe down into individual seasons
def organize_by_season(seasons, df):
    df_orig = df
    nhl_dfs = []
    for season in seasons:
        df = df_orig.copy()
        df["game"] = df["game"].query(f"season == {season}")
        # filter games to just one season
        # when we call df, we are actually calling the keys in the dict of df and this is why we can now call df[]as opposed to df_game....
        for name in ["game_skater_stats", "game_plays", "game_shifts"]:
            # do an inner merge to reduce the number of rows...keeping only the rows where game and game_id match ....
            df[name] = pd.merge(df[name], df["game"][["game_id"]], on="game_id")
            for key, val in df.items():
                print(f"{key:>25}: {len(val)}")
        # reduce df['game_plays'] df in advance
        cols = ["play_id", "game_id", "team_id_for", "event", "time"]
        events = ["Shot", "Blocked Shot", "Missed Shot", "Goal"]
        # using .loc here as a mask
        df["game_plays"] = df["game_plays"].loc[df["game_plays"]["event"].isin(events)]
        # defining "time" col
        df["game_plays"]["time"] = (
            df["game_plays"]["periodTime"] + (df["game_plays"]["period"] - 1) * 1200
        )
        df["game_plays"] = df["game_plays"][cols]

        print(f"reduced game_plays num rows: {len(df['game_plays'])}")

        # initialize corsi df
        # sort all rows by game_id and on ties defer to player_id... everything with the same game_id will be grouped together
        df_corsi = df["game_skater_stats"].sort_values(["game_id", "player_id"], ignore_index=True)[
            ["game_id", "player_id", "team_id"]
        ]

        nhl_dfs.append([season, create_corsi_stats(df_corsi, df)])


    return nhl_dfs

In [4]:
# Takes a list of pandas dataframes, calculates corsi statistics and adds them to dataframes
def create_corsi_stats(df_corsi, df):
    df_corsi[["CF", "CA", "C"]] = np.nan

    game_id_prev = None
    t1 = perf_counter()
    for i, row in df_corsi.iterrows():
        game_id, player_id, team_id = row.iloc[:3]
        if i % 1000 == 0:
            print(f"{i:>6}/{len(df_corsi)}, {perf_counter() - t1:.2f} s")
        if game_id != game_id_prev:
            shifts_game = df["game_shifts"].query(f"game_id == {game_id}")
            plays_game = df["game_plays"].query(f"game_id == {game_id}")
        shifts_player = shifts_game.query(f"player_id == {player_id}")
        mask = (
            shifts_game["shift_start"].searchsorted(plays_game["time"])
            - shifts_game["shift_end"].searchsorted(plays_game["time"])
        ).astype(bool)
        plays_player = plays_game[mask]
        # mask was it for or against our team. is it for team of the player whose player_id we are looking at
        is_our_team = plays_player["team_id_for"] == team_id
        is_missed_shot = plays_player["event"] == "Missed Shot"
        CF = (is_our_team ^ is_missed_shot).sum()
        # number of rows in the df
        CA = len(plays_player) - CF
        C = CF - CA
        df_corsi.iloc[i, 3:] = [CF, CA, C]
    df_corsi["CF_Percent"] = df_corsi["CF"]/(df_corsi["CF"] + df_corsi["CA"])

    return df_corsi

In [5]:
# Writes csv files for individual NHL seasons from a list of pandas dataframes
def write_csv(dfs):
    for df in dfs:
        df[1].to_csv(f"corsi_vals/Corsi_{df[0]}.csv")

In [6]:
df_master = load_data()
seasons = [20152016, 20162017, 20172018, 20182019, 20192020]

load
        game_skater_stats: 0.5391 sec, 853404 rows
               game_plays: 8.838 sec, 4217063 rows
              game_shifts: 2.162 sec, 9900705 rows
                     game: 0.02843 sec, 23735 rows


In [7]:
nhl_dfs = organize_by_season(seasons, df_master)

        game_skater_stats: 47553
               game_plays: 4217063
              game_shifts: 9900705
                     game: 1321
        game_skater_stats: 47553
               game_plays: 413156
              game_shifts: 9900705
                     game: 1321
        game_skater_stats: 47553
               game_plays: 413156
              game_shifts: 1039022
                     game: 1321
reduced game_plays num rows: 144387
     0/47553, 0.00 s
  1000/47553, 2.49 s
  2000/47553, 4.98 s
  3000/47553, 7.48 s
  4000/47553, 10.06 s
  5000/47553, 12.93 s
  6000/47553, 15.49 s
  7000/47553, 17.96 s
  8000/47553, 20.54 s
  9000/47553, 23.15 s
 10000/47553, 25.81 s
 11000/47553, 28.28 s
 12000/47553, 30.79 s
 13000/47553, 33.35 s
 14000/47553, 35.95 s
 15000/47553, 38.44 s
 16000/47553, 40.94 s
 17000/47553, 43.42 s
 18000/47553, 45.99 s
 19000/47553, 48.72 s
 20000/47553, 51.31 s
 21000/47553, 53.92 s
 22000/47553, 56.48 s
 23000/47553, 59.00 s
 24000/47553, 61.62 s
 25000/47553, 6

In [8]:
write_csv(nhl_dfs)